# PyTorch qGAN Implementation

Description

adapted from [PyTorch GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py)

In [1]:
# Necessary imports

import numpy as np

from torch import Tensor, stack, reshape
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F


from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import Gradient, StateFn
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.datasets.dataset_helper import discretize_and_truncate

# Set seed for random generators
algorithm_globals.random_seed = 42

- m samples - Gradient Lin Comb
- generic loss functions? for qGANs.
- Parameter from PyTorch
- PyTorch Discriminator, Loss
- Generator as samples to PyTorch Discriminator - Tensor? with custom gradient
- for grad qnn.backward(sampling=False, sparse=True)

### Load training data

In [2]:
data_dim = [2, 2]

training_data = np.random.default_rng().multivariate_normal(mean=[0., 0.], cov=[[1, 0], [0, 1]], size=1000, check_valid='warn',
                                                        tol=1e-8, method='svd')


batch_size = 100

dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

In [3]:
# declare quantum instance
backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend, shots = batch_size)

In [4]:
bounds_min = np.percentile(training_data, 5, axis=0)
bounds_max = np.percentile(training_data, 95, axis=0)
bounds = []
for i, _ in enumerate(bounds_min):
    bounds.append([bounds_min[i], bounds_max[i]])

(data,
data_grid,
grid_elements,
prob_data ) = discretize_and_truncate(
training_data,
np.array(bounds),
data_dim,
return_data_grid_elements=True,
return_prob=True,
prob_non_zero=True,
)

In [5]:
def generator_(qnn, parameters):
    """

    """
    circuit_qnn = CircuitQNN(qnn, input_params=[], weight_params = parameters,
                                 quantum_instance=qi, sampling=True, sparse=False,
                                 interpret=lambda x: grid_elements[x]) # gradient=Gradient(), input_gradients=True,

    return TorchConnector(circuit_qnn)

def generator_grad(qnn, parameters, param_values, grad_method = 'param_shift'):
    grad = Gradient(grad_method=grad_method).gradient_wrapper(StateFn(qnn), parameters, backend=qi)
    grad_values = grad(param_values)
    return grad_values.tolist()

### Definition of classical discriminator

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.Linear_in = nn.Linear(len(data_dim), 51)
        self.Leaky_ReLU = nn.LeakyReLU(0.2, inplace=True)
        self.Linear51 = nn.Linear(51, 26)
        self.Linear26 = nn.Linear(26, 1)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, input):
        x = self.Linear_in(input)
        x = self.Leaky_ReLU(x)
        x = self.Linear51(x)
        x = self.Leaky_ReLU(x)
        x = self.Linear26(x)
        x = self.Sigmoid(x)
        return x

### QNN Definition

In [7]:
qnn = QuantumCircuit(sum(data_dim))
qnn.h(qnn.qubits)
ansatz = TwoLocal(sum(data_dim), "ry", "cz", reps=2, entanglement="circular")
qnn.compose(ansatz, inplace=True)

### Definition of the loss functions

In [8]:
# Loss function
g_loss_fun = nn.BCELoss()
d_loss_fun = nn.BCELoss()

### Custom gradients for the generator loss function

In [9]:
def g_loss_fun_grad(param_values, discriminator_):
    """

    """
    grads = generator_grad(param_values)
    loss_grad = ()
    for j, grad in enumerate(grads):
        cx = grad[0].tocoo()
        input = []
        target = []
        weight = []
        for index, prob_grad in zip(cx.col, cx.data):
            input.append(grid_elements[index])
            target.append([1.])
            weight.append([prob_grad])
        bce_loss_grad = F.binary_cross_entropy(discriminator_(Tensor(input)), Tensor(target), weight=Tensor(weight))
        loss_grad += (bce_loss_grad, )
    loss_grad = stack(loss_grad)
    return loss_grad

class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

### Definition of the optimizers

In [10]:
# Initialize generator and discriminator
generator = generator_()
discriminator = Discriminator()

lr=0.0002
b1=0.5
b2=0.999
n_epochs=100

optimizer_G = Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

TypeError: generator_() missing 2 required positional arguments: 'qnn' and 'parameters'

### Training

In [ ]:
for epoch in range(n_epochs):
    for i, data in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(data.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(data.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_data = Variable(data.type(Tensor))
        # Generate a batch of images
        gen_data = generator()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        disc_data = discriminator(real_data)
        real_loss = d_loss_fun(disc_data, valid)

        fake_loss = d_loss_fun(discriminator(gen_data), fake)  # (discriminator(gen_data).detach(), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward(retain_graph=True)
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # # Loss measures generator's ability to fool the discriminator
        g_loss = g_loss_fun(discriminator(gen_data), valid)
        g_loss.retain_grad = True
        g_loss_grad = g_loss_fun_grad(generator.weight.data.numpy(), discriminator)
        g_loss.backward(retain_graph=True) # TODO gradient=Tensor([1.])
        for j, param in enumerate(generator.parameters()):
            param.grad = g_loss_grad
        optimizer_G.step()


        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        # if batches_done % optimizer_G.sample_interval == 0:
        #     #TODO: Do something like storing, printing
        #     pass

### Open Questions

- Why do we want to support a sparse representation? The output of the quantum generator needs to have the same dimension as the input of the classical discriminator. If the dimensions are larger than what a classical representation can handle then we anyways have a problem.